In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import time
import openpyxl

### 엑셀 파일 생성하기
파일명, 시트명, 컬럼명을 정해줍니다.

In [8]:
#엑셀파일 생성
file_path = "파일명.xlsx"
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "시트명"       

column_names = ['브랜드','상품명','카테고리','정가','할인가','아이디','별점','피부정보','피부타입','피부고민','자극도']
ws.append(column_names)
wb.save(file_path)

In [9]:
import os

#크롬 드라이버 자동 업데이트
from webdriver_manager.chrome import ChromeDriverManager

# 브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option('detach', True)

# 불필요한 에러 메세지 없애기
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

# 크롬 드라이버 경로 가져오기
driver_path = ChromeDriverManager().install()

# 크롬 드라이버 생성
driver = webdriver.Chrome(service=Service(driver_path), options=chrome_options)

### 크롤링 함수 만들기
제품 상세 페이지에서 상단 제품정보(브랜드명,상품명,카테고리,정가,할인가)와  
하단의 리뷰고객 좌픅정보(아이디,별점,피부정보)와 우측정보(피부타입,피부고민,자극도)를 가져옵니다.  
위의 데이터를 엑셀에 계속 추가하는 함수를 만듭니다. 

In [4]:
def customer_info():

    for j in range(1,11):

        #브랜드
        try:
            brand = driver.find_element(By.XPATH, '//*[@id="moveBrandShop"]').text
        except:
            brand ="없음"

        #상품명
        try:
            p_name = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[2]/div/p[2]').text
        except:
            p_name="없음"

        #카테고리
        try:
            p_category = driver.find_element(By.XPATH, '//*[@id="dtlCatNm"]').text
        except:
            p_category="없음"

        #정가
        try:
            price =driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[2]/div/div[1]/span[1]').text
        except:
            price=0

        #할인가
        try:
            discount = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[2]/div/div[1]/span[2]').text
        except:
            discount=0

        #고객 아이디
        try:
            _id = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{j}]/div[1]/div/p[1]/a[1]').text
        except:
            _id ="없음"

        #별점
        try:
            _star = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({j}) > div.review_cont > div.score_area > span.review_point > span').text
        except:
            _star="없음"

        #고객 피부 정보
        try:
            _info = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({j}) > div.info > div > p.tag').text
        except:
            _info = "없음"


        #피부 타입
        try:
            skin_type = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({j}) > div.review_cont > div.poll_sample > dl:nth-child(1) > dd > span').text
        except:
            skin_type = "없음"
            
        #피부 고민
        try:
            skin_trouble = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({j}) > div.review_cont > div.poll_sample > dl:nth-child(2) > dd > span').text
        except:
            skin_trouble = "없음"
        #자극도
        try:
            skin_irritation = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({j}) > div.review_cont > div.poll_sample > dl:nth-child(3) > dd > span').text
        except:
            skin_irritation = "없음"
            
        #엑셀 데이터 쌓기
        ws.append([brand,p_name,p_category,price,discount,_id, _star, _info, skin_type, skin_trouble, skin_irritation])
        time.sleep(1)            


### 웹페이지 해당 주소로 이동하여 크롤링함수를 실행하는 반복문을 만듭니다.
반복문에 들어가야 하는 것  
1. 리뷰버튼 클릭  
2. 화면 맨 아래까지 스크롤하기 (코드제공)
3. 10페이지 이하일 때 : 10페이지 크롤링 한 후 다음페이지 화살표 버튼 누르기 , 마지막 페이지인 경우 last_page==True
4. 11페이지부터 규칙을찾아 다음페이지로 이동하게 만들기, 마지막 페이지인 경우 last_page==True

In [10]:
driver.implicitly_wait(5)  #웹페이지 로딩 될때까지 5초는 기다림  
driver.maximize_window()   #화면 최대화
driver.get("상품 웹페이지 링크를 첨부") 
time.sleep(3)


#리뷰버튼 클릭
rv = driver.find_element(By.XPATH, '//*[@id="reviewInfo"]/a')
rv.click()
time.sleep(3)

#리뷰 하단 끝까지 스크롤하는 함수 (빈칸없음.그대로 코드 사용가능)
before_h = driver.execute_script("return window.scrollY")         #스크롤 전 높이
#화면 맨아래까지 스크롤
while True:
    driver.find_element(By.CSS_SELECTOR,"body").send_keys(Keys.END)

    time.sleep(1)

    #스크롤 후 높이
    after_h = driver.execute_script("return window.scrollY")

    #스크롤 값이 같으면 스크롤 멈춤
    if after_h == before_h:
        break
    before_h = after_h   

        
        
last_page=False
    
for k in range(1,101):  #100페이지 크롤링 한다고 했을 때 (상품당 최대 100페이지까지 있음)

    #마지막 페이지면 종료
    if last_page == True:
        break

    #페이지 숫자 10이하 일 때
    if k<11:
        try:  
            if k != 10:
                customer_info()
                next_page_button=driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({k+1})')
                next_page_button.click()
                time.sleep(3)


            elif k == 10:          #10페이지 크롤링 한 후에 다음페이지로 가는 화살표 버튼 클릭
                customer_info()
                next_page_button = driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next')
                wait = WebDriverWait(driver, 10)  # 최대 10초 동안 대기
                next_page_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next')))
                next_page_button.click()
                time.sleep(3)
                    
        except:
            last_page = True
                   
                    
    #페이지 숫자 11이상 일 때  (규칙을 찾아 각 페이지 크롤링 후 다음 페이지로 이동하도록 코드 작성)        
    elif k>10 :
        try:
            if k%10 !=0:
                customer_info()
                next_page_button=driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({(k%10)+2})')
                next_page_button.click()
                time.sleep(3)
            

            elif k%10 ==0:
                customer_info()
                next_page_button = driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next')
                wait = WebDriverWait(driver, 10)  # 최대 10초 동안 대기
                next_page_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next')))
                next_page_button.click()
                time.sleep(3)

        except:
            last_page = True  


### 크롤링한 결과를 엑셀에 저장 (상단에서 만든 엑셀 파일명고 동일하게)

In [11]:
try:
    wb.save(file_path)
    print("엑셀 파일이 성공적으로 저장되었습니다.")
except Exception as e:
    print("엑셀 파일 저장 중 오류 발생:", e)    

엑셀 파일이 성공적으로 저장되었습니다.
